# **사전 준비**

In [ ]:
!pip install langchain langchain_openai langchain_community pypdf faiss-cpu

In [ ]:
from google.colab import drive
import os

# 먼저 구글 드라이브 마운트
drive.mount('/content/drive')

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")

# 환경 변수에서 API 키 가져오기
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = (
    "/content/drive/MyDrive/langchain-tutorial/Ch04. Advanced Rag/Data/투자설명서.pdf"
)
loader = PyPDFLoader(file_path)

doc_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap = 100)

docs = loader.load_and_split(doc_splitter)

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

# 데이터를 임베딩으로 변환
embedding = OpenAIEmbeddings(model="text-embeddings-3-large")

In [ ]:
# FAISS 라이브러리 임포트
from langchain_community.vectorstores import FAISS

# FAISS 벡터 스토어 생성
faiss_store = FAISS.from_documents(docs, embedding)

# FAISS 벡터 스토어 저장
persist_directory = "/content/DB"
faiss_store.save_local(persist_directory)

In [ ]:
# 저장한 FAISS DB 불러오기
vectordb = FAISS.load_local(persist_directory, embeddings=embedding, allow_dangerous_deserializtion=True)

In [ ]:
from pydantic import BaseModel, Field
from langchain import PromptTemplate
from langchain.docsstore.document import Document
from typing import List, Dict, Any, Tuple
from langchain_openai import ChatOpenAI
from textwrap import dedent
from langchain_core.output_parsers import JsonOutputParser

# **함수 정의**

In [ ]:
class RelevanceScore(BaseModel):
  relevance_score: float = Field(description="문서가 쿼리와 얼마나 관련이 있는지를 나타내는 점수.")

def reranking_documents(query: str, docs: List[Document], top_n: int = 2) ->
List[Document]:
  parser = JsonOutputParser(pydantic_object=RelevanceScore)
  human_message_prompt = PromptTemplate(
      template = """
      1점부터 10점까지 점수를 매겨, 다음 문서가 질문이 얼마나 관련이 있는지 평가해주세요.
      단순히 키워드가 일치하는 것이 아니라 쿼리의 구체적인 맥락과 의도를 고려하세요.
      {format_instructions}
      question: {query}
      document: {doc}
      relevance_score:""",
      input_variables=["query", "docs"],
      partial_variables={"format_instructions": parser.get_format_instructions()}
  )

  llm = ChatOpenAI(temparature=0, model_name="gpt-4o", max_tokens=3000)
  chain = human_message_prompt | llm | parser
  scored_docs = []

  for doc in docs:
    input_data = {"query": query, "doc": doc.page_content}

    try:
      score = chain.invoke(input_data)['relevance_score']
      score = float(score)
    except Exception as e:
      print(f"오류 발생: {srt(e)}")
      # 기본 점수를 5점으로 설정
      default_score = 5
      print(f"기본 점수 {default_score}점을 사용합니다.")
      score = default_score

    scored_docs.append((doc, score))

  reranked_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)

  return [doc for doc, _ in reranked_docs[:top_n]]

# **문서 리랭킹**

In [ ]:
query = "이 회사의 2022년 영업손실이 정확히 얼마야?"
initial_docs = vectordb.similarity_search(query, k=4)
reranked_docs = reranking_documents(query, initial_docs)

In [ ]:
# 4개의 초기 검색 결과 출력
print(f"Query: {query}\n\n")

print("Top initial documents:")
for i, doc in enumerate(initial_docs):
  print(f"\nDocument {i+1}:")
  print(doc.page_content)

# 리랭킹 결과 출력
print("\n\nTop reranked documents:")
for i, doc in enumerate(reranked_docs):
  print(f"\nDocument {i+1}:")
  # 각 문서 출력
  print(doc.page_content)

# **리랭킹 RAG**

In [ ]:
from langchain_core.retrievers import BaseRetriever
from langchain.chains import RetrievalQA

# CustomRetriever 체인을 생성
class CustomRetriever(BaseRetriever, BaseModel):
  vectorstore: Any = Field(description="Retriaval을 위한 벡터 저장소")

  class Config:
    arbitrary_types_allowed = True

  # num_docs 파라미터로 리랭킹 후 반환할 최종 문서의 수를 정의
  def get_relevant_documents(self, query:str, num_docs=2) -> List[Document]:
    initial_docs = self.vectorstore.similarity_search(query, k=4)

    return reranking_documents(query, initial_docs, top_n=num_docs)

# **체인 구성**

In [ ]:
# CustomRetriever 인스턴스 생성
custom_retriever = CustomRetriever(vectorstore=vectordb)

# 답변용 LLM 인스턴스 생성
llm = ChatOpneAI(temperature=0.2, model_name="gpt-4o")

# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever,
    return_source_documents=True
)

In [ ]:
qa_chain.invoke("이 회사의 2022년 영업손실이 정확히 얼마야?")

# **체인 구성**

In [ ]:
# CustomRetriever 인스턴스 생성
custom_retriever = CustomRetriever(vectorstore=vectordb)

# 답변용 LLM 인스턴스 생성
llm = ChatOpneAI(temperature=0.2, model_name="gpt-4o")

# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever,
    return_source_documents=True
)

In [ ]:
qa_chain.invoke("이 회사의 2022년 영업손실이 정확히 얼마야?")

# **체인 구성**

In [ ]:
# CustomRetriever 인스턴스 생성
custom_retriever = CustomRetriever(vectorstore=vectordb)

# 답변용 LLM 인스턴스 생성
llm = ChatOpneAI(temperature=0.2, model_name="gpt-4o")

# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever,
    return_source_documents=True
)

In [ ]:
qa_chain.invoke("이 회사의 2022년 영업손실이 정확히 얼마야?")

# **체인 구성**

In [ ]:
# CustomRetriever 인스턴스 생성
custom_retriever = CustomRetriever(vectorstore=vectordb)

# 답변용 LLM 인스턴스 생성
llm = ChatOpneAI(temperature=0.2, model_name="gpt-4o")

# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever,
    return_source_documents=True
)

In [ ]:
qa_chain.invoke("이 회사의 2022년 영업손실이 정확히 얼마야?")